In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats
import csv
from statsmodels.stats.multitest import multipletests

In [2]:
all_mean_arr=0.008131350120614151
all_std_arr=0.003374720423931657
mean_dict_different_race={'ACB': 0.043905219284227484,
 'ASW': 0.061747415774036245,
 'BEB': 0.042622549524204016,
 'CDX': 0.05629086218276262,
 'CEU': 0.03350899999290784,
 'CHB': 0.05201986349516006,
 'CHS': 0.03643899713354041,
 'CLM': 0.04224577966848677,
 'ESN': 0.03623741211034457,
 'FIN': 0.053129961026517944,
 'GBR': 0.05656052651694239,
 'GIH': 0.05157971151605565,
 'GWD': 0.03170015029229307,
 'IBS': 0.0371240314050134,
 'ITU': 0.050724822118831774,
 'JPT': 0.0517456479295656,
 'KHV': 0.04564560273080832,
 'LWK': 0.04973674801295715,
 'MSL': 0.04960771674064523,
 'MXL': 0.05377595974774834,
 'PEL': 0.045806236523102895,
 'PJL': 0.040244478916681585,
 'PUR': 0.04003893507415216,
 'STU': 0.048497323483452925,
 'TSI': 0.05007660537347943,
 'YRI': 0.031315221824936575}
std_dict_different_race={'ACB': 0.0035837115206611876,
 'ASW': 0.0044435636072078216,
 'BEB': 0.003260312673626174,
 'CDX': 0.00391015689396564,
 'CEU': 0.002766896630079425,
 'CHB': 0.0037059827211937194,
 'CHS': 0.002921456474220746,
 'CLM': 0.003272057962121683,
 'ESN': 0.0031803272910866064,
 'FIN': 0.0038008600285356704,
 'GBR': 0.003936807447491629,
 'GIH': 0.0037221796184063703,
 'GWD': 0.0029073393803639483,
 'IBS': 0.0029858040112478645,
 'ITU': 0.003597951675686167,
 'JPT': 0.003708599258289701,
 'KHV': 0.00343805410624286,
 'LWK': 0.003848934277722462,
 'MSL': 0.003934844393069718,
 'MXL': 0.003865357254429113,
 'PEL': 0.003392906748737866,
 'PJL': 0.003126523821241254,
 'PUR': 0.0031949284883587385,
 'STU': 0.00353413807778532,
 'TSI': 0.0036716230350532964,
 'YRI': 0.002910364879924007}

In [3]:
import re
def getinteger(s):
    match = re.search(r'chr(\d+)', s)

    if match:
        return match.group(1) 
    
getinteger('chr6'.split("_")[0])

'6'

In [4]:
mhc_race_positions_dict={'All':{'mhc':[[32589647, 32751796]]}}

In [5]:
split_race_region={'ACB': {'chr2': [[92299419, 92523478]],
  'chr3': [[89474731, 90288204]],
  'chr5': [[62144306, 62605777]],
  'chr14': [[105863438, 106259905]],
  'chr16': [[36332499, 46403568]],
  'chr18': [[19764009, 20828994]],
  'chr22': [[22691667, 22898645]]},
  'All':{'chr2': [[108377615, 108667023]],
 'chr3': [[87292011, 87555666]],
 'chr9': [[88434984, 88700396]],
 'chr11': [[39680311, 39930646]],
 'chr12': [[20836055, 21097074], [34328866, 34516639]],
 'chr14': [[31627550, 31888106]]},
 'ASW': {'chr2': [[92299419, 92507872],
   [92633551, 92902449],
   [93680283, 94160185],
   [94892770, 95546698],
   [202715454, 203549066]],
  'chr3': [[93559360, 94116184]],
  'chr8': [[49383975, 49798135], [103467484, 103900956]],
  'chr10': [[72960046, 73409340]],
  'chr14': [[105859287, 106198900]],
  'chr16': [[36260059, 46728534]]},
 'BEB': {'chr15': [[77138940, 77566277]]},
 'CDX': {'chr8': [[70075248, 70483016]],
  'chr11': [[50311095, 50728899]],
  'chr14': [[66379055, 66778338]],
  'chr20': [[21834643, 22233777]]},
 'CEU': {'chr8': [[84442692, 84821607]],
  'chr12': [[34328866, 34590462]],
  'chr14': [[105917600, 106259905]],
  'chr22': [[22661982, 22898645]]},
 'CHB': {'chr6_partC': [[85501992, 86104633]],
  'chr11': [[47086853, 47921826]]},
 'CHS': {'chr10': [[63107761, 63493252]],
  'chr13': [[47866045, 48251299]],
  'chr15': [[76471437, 76909646]]},
 'CLM': {'chr19': [[42860739, 43046093]]},
 'ESN': {'chr12': [[72479117, 72899345]]},
 'FIN': {'chr3': [[75380511, 75596024]],
  'chr5': [[131238500, 132001329]],
  'chr17': [[39204905, 39592604], [45610677, 46138242]]},
 'GBR': {'chr19': [[22622704, 23047137]], 'chr22': [[41329774, 41808259]]},
 'GIH': {'chr3': [[157950988, 158348010]],
  'chr6_partC': [[85696088, 86044297]],
  'chr12': [[132935977, 133264749]],
  'chr19': [[42820338, 43026385]]},
 'GWD': {'chr2': [[213229719, 213686213]],
  'chr3': [[47078886, 47548303]],
  'chr10': [[38164827, 38614638]]},
 'IBS': {'chr3': [[75380511, 75596024]]},
 'ITU': {'chr1': [[189001539, 189336299]],
  'chr8': [[50652103, 51059321]],
  'chr10': [[72260319, 72661979]],
  'chr12': [[32954541, 33301513]]},
 'JPT': {'chr10': [[73001439, 73409340]], 'chr14': [[66187582, 66778338]]},
 'KHV': {'chr2': [[186057591, 186516143]],
  'chr4': [[151301085, 151752781]],
  'chr10': [[38073145, 38635036], [73001439, 73409340]],
  'chr17': [[58394956, 59118140]]},
 'LWK': {'chr2': [[93680283, 94160185]],
  'chr9': [[64439637, 65054326]],
  'chr10': [[102847077, 103283225]],
  'chr14': [[105863438, 106385732]],
  'chr16': [[36332499, 46403568]],
  'chr22': [[22691667, 22898645]]},
 'MSL': {'chr2': [[93680283, 94143754]],
  'chr5': [[46140397, 46434522]],
  'chr10': [[38073145, 38635036]],
  'chr14': [[105863438, 106259905]],
  'chr18': [[19764009, 20828994]],
  'chr19': [[27564615, 27844019]],
  'chr20': [[18364231, 18744031]],
  'chr22': [[22691667, 22898645]]},
 'MXL': {'chr3': [[94780047, 95118014]], 'chr11': [[49588569, 50709669]]},
 'PEL': {'chr4': [[150649663, 151256009]],
  'chr5': [[46140397, 46434522]],
  'chr11': [[48330435, 49133873]]},
 'PJL': {'chr6_partC': [[118358152, 118798801]],
  'chr7': [[57291163, 57656843]]},
 'PUR': {'chr11': [[49133908, 50818091], [54704427, 55220212]]},
 'STU': {'chr2': [[135176374, 135572114]],
  'chr3': [[23111879, 23593852]],
  'chr14': [[66143522, 66818672]]},
 'TSI': {'chr2': [[193268274, 193813343]], 'chr13': [[55278837, 55753073]]},
 'YRI': {'chr10': [[63150067, 63541883]],
  'chr14': [[66379055, 66778338]],
  'chr22': [[22691667, 22898645]]}}

In [6]:
allrecords=[]

In [7]:
df_mhc_all=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles/posmhc.csv")

In [8]:
for race,values in mhc_race_positions_dict.items():
    if values!={}:
        if race=="All":
            for region in values["mhc"]:
                onerecord=[]
                onerecord+=["All",6,1]
                top_1000_P_list=df_mhc_all[(df_mhc_all["Unnamed: 0"]>=region[0])&(df_mhc_all["Unnamed: 0"]<=region[1])]["P"].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P, all_mean_arr, all_std_arr)),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)

In [9]:
for race,values in split_race_region.items():
    if race=="All":
        for k, v in values.items():
            chr=getinteger(k)
            # filename="pos_prob_"+k+".csv"
            df=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_csvfiles_inte/"+k+".csv")
            for region in v:
                onerecord=[]
                onerecord+=["All",chr,0]
                top_1000_P_list=df[(df["Unnamed: 0"]>=region[0])&(df["Unnamed: 0"]<=region[1])]["P"].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P, all_mean_arr, all_std_arr)),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)
    else:
        for k,v in values.items():
            chr=getinteger(k)
            #filename=k.split("/")[0]+"_"+k.split("/")[1]+".csv"
            df=pd.read_csv("/data2/wangxuedong/mhc_test_data/first_review/similar_region_race_integration/"+k+".csv")
            for region in v:
                onerecord=[]
                onerecord+=[race,chr,0]
                top_1000_P_list=df[(df["Unnamed: 0"]>=region[0])&(df["Unnamed: 0"]<=region[1])][race].to_list()
                # sortedlist=sorted(top_1000_P_list,reverse=True)
                # top_1000=sortedlist[:1000]
                mean_1000_P=np.mean(top_1000_P_list)
                std_1000_P=np.std(top_1000_P_list, ddof=1)
                pvalue=round((1-stats.norm.cdf(mean_1000_P,mean_dict_different_race[race], std_dict_different_race[race])),15)
                onerecord+=[mean_1000_P,std_1000_P,pvalue,region[0],region[1],region[1]-region[0]]
                allrecords.append(onerecord)

In [10]:
#csv1
columns=["Race","chr","isMHC","Average of P","Std of P","P-value","start","end","len"]

In [11]:
#generatedf
def generatedf(columns,allrecords):
    dictforDF=dict()
    for i in range(len(columns)):
        midarr=[]
        for record in allrecords:           
            midarr.append(record[i])
        dictforDF[columns[i]]=midarr
    return dataframe(dictforDF)





In [12]:
dfnew=generatedf(columns=columns,allrecords=allrecords)

In [13]:
dfnew

,Race,chr,isMHC,Average of P,Std of P,P-value,start,end,len
0,All,6,1,0.031347,0.031433,3.005000e-12,32589647,32751796,162149
1,ACB,2,0,0.050331,0.009406,3.647303e-02,92299419,92523478,224059
2,ACB,3,0,0.051636,0.014171,1.549558e-02,89474731,90288204,813473
3,ACB,5,0,0.051499,0.021974,1.704550e-02,62144306,62605777,461471
4,ACB,14,0,0.052810,0.015259,6.482087e-03,105863438,106259905,396467
...,...,...,...,...,...,...,...,...,...
93,TSI,2,0,0.060532,0.025885,2.202584e-03,193268274,193813343,545069
94,TSI,13,0,0.061380,0.030473,1.040033e-03,55278837,55753073,474236
95,YRI,10,0,0.036521,0.012173,3.683454e-02,63150067,63541883,391816
96,YRI,14,0,0.037637,0.012012,1.492358e-02,66379055,66778338,399283


In [14]:
dfnew.to_csv("tables2.csv")

In [1]:
import pandas as pd

In [2]:
dfnew=pd.read_csv('tables2.csv')

In [5]:
dfnew = dfnew.drop('Unnamed: 0', axis=1)

In [6]:
dfnew

,Race,chr,isMHC,Average of P,Std of P,P-value,start,end,len
0,All,6,1,0.031347,0.031433,3.005000e-12,32589647,32751796,162149
1,ACB,2,0,0.050331,0.009406,3.647303e-02,92299419,92523478,224059
2,ACB,3,0,0.051636,0.014171,1.549558e-02,89474731,90288204,813473
3,ACB,5,0,0.051499,0.021974,1.704550e-02,62144306,62605777,461471
4,ACB,14,0,0.052810,0.015259,6.482087e-03,105863438,106259905,396467
...,...,...,...,...,...,...,...,...,...
93,TSI,2,0,0.060532,0.025885,2.202584e-03,193268274,193813343,545069
94,TSI,13,0,0.061380,0.030473,1.040033e-03,55278837,55753073,474236
95,YRI,10,0,0.036521,0.012173,3.683454e-02,63150067,63541883,391816
96,YRI,14,0,0.037637,0.012012,1.492358e-02,66379055,66778338,399283


In [7]:

pvalue_list=dfnew["P-value"].to_list()
pvalue_list

[3.005e-12,
 0.036473034288754,
 0.015495580636593,
 0.017045501800748,
 0.006482086550005,
 0.016593754510368,
 0.003057344696495,
 0.001770386163723,
 0.01408193822047,
 0.000282667061771,
 0.002251218339173,
 0.000708794678935,
 3.168e-12,
 0.007085479288401,
 0.001051466384711,
 0.033032862542107,
 0.03472666243834,
 0.030740142034276,
 0.031214788280073,
 0.024823384271637,
 0.0330901916592,
 0.007431412591712,
 9.6489537466e-05,
 0.021465894492756,
 0.022843007078355,
 0.025838695664943,
 0.015320688470564,
 0.020205681636067,
 0.028672569280631,
 0.005235558968814,
 0.019267384840117,
 0.016661700692924,
 0.037118544859958,
 0.011780636420527,
 1.8697836984e-05,
 0.001893670049747,
 0.008462451612533,
 0.023597383069861,
 0.006870970743808,
 0.021442329862059,
 0.010369950885265,
 0.009247615445104,
 0.000312786221859,
 0.013779167017559,
 0.014385173467505,
 0.00527117521801,
 0.014382258230871,
 0.001227422226354,
 0.005368669948325,
 0.002925217673836,
 0.001256338726583,
 0.

In [9]:
from statsmodels.stats.multitest import multipletests
pvalue_list=dfnew["P-value"].to_list()
corr_pvals = multipletests(pvalue_list, method='fdr_bh')[1]
corr_pvals

array([1.55232000e-10, 3.71185449e-02, 2.31476172e-02, 2.33534139e-02,
       1.46528053e-02, 2.33263810e-02, 9.36311813e-03, 8.17108879e-03,
       2.27378548e-02, 3.40589442e-03, 8.17108879e-03, 6.31471623e-03,
       1.55232000e-10, 1.47739781e-02, 7.73157850e-03, 3.46789719e-02,
       3.58232939e-02, 3.38486957e-02, 3.39894361e-02, 2.93095381e-02,
       3.46789719e-02, 1.51724674e-02, 1.57599578e-03, 2.62957208e-02,
       2.76372184e-02, 2.97904962e-02, 2.31476172e-02, 2.59137705e-02,
       3.21386351e-02, 1.31532414e-02, 2.57929600e-02, 2.33263810e-02,
       3.71185449e-02, 2.09909522e-02, 4.58097006e-04, 8.17108879e-03,
       1.65864052e-02, 2.82017505e-02, 1.46528053e-02, 2.62957208e-02,
       1.88195405e-02, 1.77699277e-02, 3.40589442e-03, 2.27378548e-02,
       2.27378548e-02, 1.31532414e-02, 2.27378548e-02, 7.73157850e-03,
       1.31532414e-02, 9.24746232e-03, 7.73157850e-03, 1.40697021e-02,
       2.33263810e-02, 2.96688434e-02, 1.46528053e-02, 2.60215446e-05,
      

In [10]:
dfnew = dfnew.assign(corr_pvals=corr_pvals)
dfnew.to_csv("tables2.csv")